In [ ]:
# Setup, Version check and Common imports

# Python ≥3.8 is required
import sys
assert sys.version_info >= (3, 5)


# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers

# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:

# Download a dataset containing pictures of cats and dogs
# Method get_file() downloads and extracts information from the zip file:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file
# The information is placed in the datasets folder of the current directory (confirm the division in sub-folders)

import os

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True, cache_dir=os.curdir)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')


In [ ]:
# Count the images

import pathlib

data_dir = pathlib.Path(PATH)

image_count = len(list(data_dir.glob('*/*/*.jpg')))
print(image_count)

In [ ]:

# Create dataset for training and validation
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
# https://www.tensorflow.org/guide/data

# Method image_dataset_from_directory() creates a Dataset object from images located in a specified directory
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory
# This method may shuffle images, adjust size and define the batch size
# This way the dataset is (almost) ready to be processed by the neural network

from tensorflow.keras.preprocessing import image_dataset_from_directory


IMG_SIZE = (160, 160)

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

train_ds = image_dataset_from_directory(train_dir, shuffle=True, seed=11, image_size=IMG_SIZE)
val_ds = image_dataset_from_directory(validation_dir, shuffle=True, seed=11, image_size=IMG_SIZE)

class_names = train_ds.class_names

train_ds = train_ds.cache().prefetch(1)
val_ds = val_ds.cache().prefetch(1)


In [ ]:
# check the dimension of the created datasets

for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

print('Class Names: ', class_names)

In [ ]:
# Datasets are very important objects and they are associated with a large set of methods
# https://www.tensorflow.org/guide/data

for m in dir(tf.data.Dataset):
    if not (m.startswith("_") or m.endswith("_")):
        func = getattr(tf.data.Dataset, m)
        if hasattr(func, "__doc__"):
            print("● {:21s}{}".format(m + "()", func.__doc__.split("\n")[0]))

In [ ]:
# Visualize a few examples

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy()/255.0) # the image from the dataset is transformed into a numpy array
        plt.title(class_names[labels[i]])
        plt.axis("off")


In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


In [ ]:
## Model_FF - FEED-FORWARD NN

# Create a feed-forward NN with Keras Sequential API: https://keras.io/api/models/

# Complete with the following architecture
# 4 hidden layers with 50 neurons each, He weight initialization and ReLU activation function
# Last hidden layer must be suitable for a classification problem with 2 classes

model_FF = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[160, 160, 3]),
    keras.layers.Rescaling(1./255),

    ### Complete the missing layers ###
])

In [ ]:
# Summary

model_FF.summary()

In [ ]:
# Model compilation
# Define the loss function (https://keras.io/api/losses/)

loss_FF = ### Add Definition here #

model_FF.compile(loss=loss_FF,
              optimizer=keras.optimizers.SGD(),
              metrics=["accuracy"])

In [ ]:
# Train for 30 epochs

history = model_FF.fit(train_ds, epochs=30, validation_data=val_ds)

In [ ]:
# Results

import pandas as pd

x = pd.DataFrame(history.history, columns = ['accuracy', 'val_accuracy'])
x.plot(figsize=(8, 5))
plt.grid(True)
plt.show()

Quiz 1: Present


1.   The last layer selected for your network
2.   The selected loss function
3.   A brief analysis of results


In [ ]:
# In the second part of the worksheet, a CNN for this problem will be created

# Before that, we will gain insight into the transformation performed by convolutional layers
# Two exampl images from the dataset are selected for the study
# In the code,  transformations are only applied to the first image
# Perform changes in the code to apply the transformation also to the second image

for images, labels in train_ds.take(1):
    im1 = images[0].numpy()/255.0
    label1 = labels[0]
    im2 = images[1].numpy()/255.0
    label2 = labels[1]


plt.figure(figsize=(10, 10))

ax = plt.subplot(1, 2, 1)
plt.imshow(im1)
plt.title(class_names[label1])
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(im2)
plt.title(class_names[label2])
plt.axis("off")




In [ ]:
# Adjust Tensor dimensions (add batch dimension)

im1 = tf.expand_dims(im1, axis=0)

im1.shape

In [ ]:
# Create vertical and random filters

# The last dimension defines the number of output channels - in this example it is just one single feature map

# You can try different filters and verify how the feature maps also change

filter1 = np.zeros(shape=(7,7,3,1), dtype = np.float32)

filter1[: , 3 , : ,] = 2
filter1[: , 1 , : ,] = -1
filter1[: , 5 , : ,] = -1

filter2 = np.random.uniform(low=0.0, high=1.0, size=(7,7,3,1))





In [ ]:
# Visualize filter1

a = filter1.squeeze()

R = a[:,:,0]
G = a[:,:,1]
B = a[:,:,2]

plt.figure(figsize=(10, 10))

ax = plt.subplot(1, 3, 1)
plt.imshow(R)
plt.title("R")
plt.axis("off")
ax = plt.subplot(1, 3, 2)
plt.imshow(G)
plt.title("G")
plt.axis("off")
ax = plt.subplot(1, 3, 3)
plt.imshow(B)
plt.title("B")
plt.axis("off")

In [ ]:
# Visualize filter2

a = filter2.squeeze()

R = a[:,:,0]
G = a[:,:,1]
B = a[:,:,2]

plt.figure(figsize=(10, 10))

ax = plt.subplot(1, 3, 1)
plt.imshow(R)
plt.title("R")
plt.axis("off")
ax = plt.subplot(1, 3, 2)
plt.imshow(G)
plt.title("G")
plt.axis("off")
ax = plt.subplot(1, 3, 3)
plt.imshow(B)
plt.title("B")
plt.axis("off")

In [ ]:
# Apply a convolutional layer with each one of the filters
# The selected methods belong to the TensorFlow library
# https://www.tensorflow.org/api_docs/python/tf/nn
# https://www.tensorflow.org/api_docs/python/tf/nn/conv2d
# Ainda não está a ser aplicada a função de ativação

output1 = tf.nn.conv2d(im1, filter1, strides=1, padding="SAME")
output2 = tf.nn.conv2d(im1, filter2, strides=1, padding="SAME")

In [ ]:
# Visualize the obtained feature maps (one for each filter)
#https://www.tensorflow.org/api_docs/python/tf/squeeze

fm1 = tf.squeeze(output1)
fm2 = tf.squeeze(output2)

plt.figure(figsize=(50, 50))

ax = plt.subplot(1, 2, 1)
plt.imshow(fm1, cmap="binary")
plt.title("Vertical", fontsize=30)
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(fm2, cmap="binary")
plt.title("Random", fontsize=30)
plt.axis("off")

# Analyze image patterns

In [ ]:
# Apply RELU activation function
# https://www.tensorflow.org/api_docs/python/tf/nn/relu

final1 = tf.nn.relu(output1)
final2 = tf.nn.relu(output2)

In [ ]:
# Visualize the obtained feature maps (one for each filter)
# https://www.tensorflow.org/api_docs/python/tf/squeeze

fmF1 = tf.squeeze(final1)
fmF2 = tf.squeeze(final2)

plt.figure(figsize=(50, 50))

ax = plt.subplot(1, 2, 1)
plt.imshow(fmF1, cmap="binary")
plt.title("Vertical", fontsize=30)
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(fmF2, cmap="binary")
plt.title("Random", fontsize=30)
plt.axis("off")

In [ ]:
# Apply MaxPool layer

pool1 = tf.nn.max_pool(final1, 2, strides=2, padding="SAME")
pool2 = tf.nn.max_pool(final2, 2, strides=2, padding="SAME")

In [ ]:
# Visualize the obtained feature maps (one for each filter)

plt.figure(figsize=(20, 20))

p1 = tf.squeeze(pool1)
p2 = tf.squeeze(pool2)

ax = plt.subplot(1, 2, 1)
plt.imshow(p1, cmap="binary")
plt.title("Vertical", fontsize=30)
plt.axis("off")
ax = plt.subplot(1, 2, 2)
plt.imshow(p2, cmap="binary")
plt.title("Random", fontsize=30)
plt.axis("off")

In [ ]:
# Model_CNN - Convolutional Neural Network

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# Create a simple CNN

# Add the final classification layer to the model

model_CNN = keras.models.Sequential([
    layers.Rescaling(1./255, input_shape=(160,160,3)),
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu',padding='same'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu',padding='same'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
     ### Complete the last layer ###
])


In [ ]:
model_CNN.summary()

In [ ]:
# Model compilation
# Define the loss function

lossCNN = ### Define the loss function ###

model_CNN.compile(loss= lossCNN,
              optimizer=keras.optimizers.SGD(),
              metrics=["accuracy"])

history = model_CNN.fit(train_ds, epochs=30, validation_data=val_ds)

In [ ]:
# Plot the evolution of the accuracy metrics

import pandas as pd

x = pd.DataFrame(history.history, columns = ['accuracy', 'val_accuracy'])
x.plot(figsize=(8, 5))
plt.grid(True)
plt.show()

Quiz 2:

1.   How many weights has each kernel/filter of the second convolutional layer?
2.   How many feature maps are generated by the last convolutional layer?
3.   What is the dimension of each one of these feature maps?

Quiz 3:

How do you analyse the results obtained by this CNN?

In [ ]:


# Design and implement one change in the CNN and repeat the training process,
# seeking for an architecture that performs more effectively.

# Among other possibilities, you might consider one of the following points:
#  1. Change the CNN architecture, adding, deleting, or changing the parameterization of convolutional, maxpooling or dense layers.
#  2. Add Batch Normalization and/or Dropout layers.
#  3. Add a callback to implement Early Stopping.

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


# Create a new model called model_CNN2


###  CODE GOES HERE   ####

